# Operowanie plikami Excela za pomocą pandas

# Korzystanie z biblioteki pandas z plikami Excela

## Studium przypadku: raportowanie w Excelu

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("sales_data/new/January.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9493 entries, 0 to 9492
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_transakcji    9493 non-null   object        
 1   sklep            9493 non-null   object        
 2   status           9493 non-null   object        
 3   data_transakcji  9493 non-null   datetime64[ns]
 4   plan             9493 non-null   object        
 5   rodzaj_umowy     9493 non-null   object        
 6   kwota            9493 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 519.3+ KB


## Odczytywanie plików Excela za pomocą pandas

In [3]:
# wybranie zakresu komórek,  które chcemy wczytać - parametry skiprows, usecols
df = pd.read_excel("xl/stores.xlsx",
                   sheet_name="2019", skiprows=1, usecols="B:F")
df

,Sklep,Pracownicy,Kierownik,Data otwarcia,Sklep firmowy
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,MISSING
2,Chicago,4,Katelin,2020-01-31,NaN
3,Boston,5,Georgiana,2017-04-01,True
4,Washington DC,3,Evan,NaT,False
5,Las Vegas,11,Paul,2020-01-06,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Sklep          6 non-null      object        
 1   Pracownicy     6 non-null      int64         
 2   Kierownik      6 non-null      object        
 3   Data otwarcia  5 non-null      datetime64[ns]
 4   Sklep firmowy  5 non-null      object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 368.0+ bytes


In [5]:
# zmiana typu danych w kolumnie sklep firmowy
def fix_missing(x):
    return False if x in ["", "MISSING"] else x

In [6]:
df = pd.read_excel("xl/stores.xlsx",
                   sheet_name="2019", skiprows=1, usecols="B:F",
                   converters={"Sklep firmowy": fix_missing})
df

,Sklep,Pracownicy,Kierownik,Data otwarcia,Sklep firmowy
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,False
2,Chicago,4,Katelin,2020-01-31,False
3,Boston,5,Georgiana,2017-04-01,True
4,Washington DC,3,Evan,NaT,False
5,Las Vegas,11,Paul,2020-01-06,False


In [7]:
# Kolumna Sklep firmowy ma teraz typ danych "bool"
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Sklep          6 non-null      object        
 1   Pracownicy     6 non-null      int64         
 2   Kierownik      6 non-null      object        
 3   Data otwarcia  5 non-null      datetime64[ns]
 4   Sklep firmowy  6 non-null      bool          
dtypes: bool(1), datetime64[ns](1), int64(1), object(2)
memory usage: 326.0+ bytes


In [8]:
# lista nazw arkuszy
sheets = pd.read_excel("xl/stores.xlsx", sheet_name=["2019", "2020"],
                       skiprows=1, usecols=["Sklep", "Pracownicy"])
sheets["2019"].head(2)

,Sklep,Pracownicy
0,New York,10
1,San Francisco,12


In [9]:
# jeśli plik żródłowy nie ma nagłówków kolumn, ustaw header=None
df = pd.read_excel("xl/stores.xlsx", sheet_name=0,
                   skiprows=2, skipfooter=3,
                   usecols="B:C,F", header=None,
                   names=["Oddział", "Liczba_pracowników", "Firmowy"])
df

,Oddział,Liczba_pracowników,Firmowy
0,New York,10,False
1,San Francisco,12,MISSING
2,Chicago,4,NaN


In [10]:
# aby jako NaN interpretowane były tylko komórki ze słowem MISSING należy użyć kombinacji na_values i keep_default_na
df = pd.read_excel("xl/stores.xlsx", sheet_name="2019",
                   skiprows=1, usecols="B,C,F", skipfooter=2,
                   na_values="MISSING", keep_default_na=False)
df

,Sklep,Pracownicy,Sklep firmowy
0,New York,10,False
1,San Francisco,12,NaN
2,Chicago,4,
3,Boston,5,True


### Menedżery kontekstu i instrukcja with

In [11]:
# utworzenie pliku 
f = open("output.txt", "w")
f.write("Jakiś tekst")
f.close()

In [12]:
with open("output.txt", "w") as f:
    f.write("Jakiś tekst")

In [14]:
with pd.ExcelFile("xl/stores.xls") as f:
    df1 = pd.read_excel(f, "2019", skiprows=1, usecols="B:F", nrows=2)
    df2 = pd.read_excel(f, "2020", skiprows=1, usecols="B:F", nrows=2)

df1

,Sklep,Pracownicy,Kierownik,Data otwarcia,Sklep firmowy
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,MISSING


In [15]:
stores = pd.ExcelFile("xl/stores.xlsx")
stores.sheet_names

['2019', '2020', '2019-2020']

In [16]:
url = ("https://raw.githubusercontent.com/fzumstein/"
       "python-for-excel/1st-edition/xl/stores.xlsx")
pd.read_excel(url, skiprows=1, usecols="B:E", nrows=2)

,Store,Employees,Manager,Since
0,New York,10,Sarah,2018-07-20
1,San Francisco,12,Neriah,2019-11-02


## Zapisywanie plików Excela za pomocą pandas

In [17]:
import numpy as np
import datetime as dt

In [18]:
data=[[dt.datetime(2020,1,1, 10, 13), 2.222, 1, True],
      [dt.datetime(2020,1,2), np.nan, 2, False],
      [dt.datetime(2020,1,2), np.inf, 3, True]]
df = pd.DataFrame(data=data,
                  columns=["Daty", "zmiennoprzecinkowe", "całkowite", "logiczne"])
df.index.name="indeks"
df

,Daty,zmiennoprzecinkowe,całkowite,logiczne
indeks,,,,
0,2020-01-01 10:13:00,2.222,1,True
1,2020-01-02 00:00:00,NaN,2,False
2,2020-01-02 00:00:00,inf,3,True


In [19]:
df.to_excel("written_with_pandas.xlsx", sheet_name="Wyjście",
            startrow=1, startcol=1, index=True, header=True,
            na_rep="<NA>", inf_rep="<INF>")

In [20]:
with pd.ExcelWriter("written_with_pandas2.xlsx") as writer:
    df.to_excel(writer, sheet_name="Arkusz1", startrow=1, startcol=1)
    df.to_excel(writer, sheet_name="Arkusz1", startrow=10, startcol=1)
    df.to_excel(writer, sheet_name="Arkusz2")